In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [ ]:
# -----------------------
# 1. Parameters
# -----------------------
beta = 0.95
c = 0.5  # per-occupant cost
kappa = 0.0
delta_b = 0.8
ubar = 0.9  # outside option / minimum utility
G = 10
Ncells = G * G
mass_total = 1.0

In [ ]:
init_pop = init_population_matrix(shape=(G, G), seed=42)

In [ ]:
# --------------------------------------------------
# 2. NEIGHBORHOOD + SITE VALUE MATRIX
# --------------------------------------------------
# Build matrix M such that S = M n (linear in n):
# S_i = n_i + 0.5 * average(neighbor n_j).
M = np.zeros((Ncells, Ncells))
for idx in range(Ncells):
    neigh = neighbors_8(idx, G)
    M[idx, idx] = 1.0
    if len(neigh) > 0:
        M[idx, neigh] = 0.5 / len(neigh)


# --------------------------------------------------
# 3. RESIDUALS FOR NONLINEAR PROGRAM
# --------------------------------------------------
def residuals(z):
    """Z = [n_1,...,n_Ncells, u]
    We want to enforce the Linear Complementarity Problem conditions:
      n_i >= 0, lambda_i >= 0, n_i * lambda_i = 0
    where lambda_i = S_i - u - c.
    - comp residual: n_i * lambda_i -> should be 0.
    - lam_neg residual: any negative part of lambda should be 0 (so penalize negatives).
    Population constraint: sum n_i = mass_total.
    We stack these into one vector for a least-squares style objective.
    """
    n = z[:Ncells]
    u = z[-1]
    S = M @ n
    lam = S - u - c  # "shadow price" of vacancy condition

    comp = n * lam  # complementarity part
    lam_neg = np.minimum(lam, 0.0)  # enforce lam >= 0 softly
    pop_res = n.sum() - mass_total  # population constraint

    return np.concatenate([comp, lam_neg, [pop_res]])


def objective(z):
    """Square the residuals and sum → scalar objective for minimize()."""
    r = residuals(z)
    return np.dot(r, r)


# --------------------------------------------------
# 4. INITIAL GUESS & BOUNDS
# --------------------------------------------------
rng = np.random.default_rng(123)
n0 = rng.dirichlet(np.ones(Ncells)) * mass_total  # random feasible starting point
u0 = 0.0
z0 = np.concatenate([n0, [u0]])

# n_i >= 0, u free
bounds = [(0.0, None)] * Ncells + [(None, None)]

# --------------------------------------------------
# 5. SOLVE THE NONLINEAR PROGRAM
# --------------------------------------------------
res = minimize(
    objective,
    z0,
    method="trust-constr",
    bounds=bounds,
    options={"maxiter": 2000, "gtol": 1e-12, "xtol": 1e-12, "verbose": 0},
)

if not res.success:
    raise RuntimeError("Solver failed: " + res.message)

z_opt = res.x
n_star = z_opt[:Ncells]
u_star = z_opt[-1]

# Clean numerical noise (tiny negatives)
n_star[n_star < 1e-12] = 0.0

# --------------------------------------------------
# 6. BACK OUT ECONOMIC OBJECTS
# --------------------------------------------------
S_star = M @ n_star  # site values
lam_star = S_star - u_star - c  # should be >=0 where n_i == 0
lam_star[lam_star < 1e-12] = 0.0

r_star = S_star - u_star  # rents from indifference condition
B_star = (c / delta_b) * n_star  # steady-state bound capital (if stationary)
pi_star = (r_star - c) * n_star - kappa * B_star  # flow profit
V_star = pi_star / (1 - beta)  # capitalized land value (stationary case)

# --------------------------------------------------
# 7. (TRIVIAL) TRANSITION PATH
# --------------------------------------------------
# With frictionless households and no shocks, you jump to steady state in one period.
T = 5
n_path = [n0.reshape(G, G)]  # initial random allocation
u_path = []
r_path = []

n_eq = n_star.reshape(G, G)
r_eq = r_star.reshape(G, G)
u_eq = u_star

for t in range(1, T + 1):
    n_path.append(n_eq)
    u_path.append(u_eq)
    r_path.append(r_eq)

# --------------------------------------------------
# 8. VACANCY / OCCUPANCY STATS
# --------------------------------------------------
threshold = 1e-10
occupied_mask = n_star > threshold
occupied_cells = occupied_mask.sum()
vacant_cells = Ncells - occupied_cells
mass_occupied = n_star[occupied_mask].sum()
mass_vacant = n_star[~occupied_mask].sum()

print(f"Occupied cells: {occupied_cells}, Vacant cells: {vacant_cells}")
print(
    f"Mass in occupied cells: {mass_occupied:.6f}, Mass in vacant cells: {mass_vacant:.6f}"
)
print(f"u*: {u_star:.6f}, mean rent: {r_star.mean():.6f}")

In [ ]:
plt.figure()
plt.imshow(n_path[0])
plt.title("Population Mass: t=0 (Random Allocation)")
plt.colorbar()
plt.xticks(range(G))
plt.yticks(range(G))
plt.tight_layout()

plt.figure()
plt.imshow(n_eq)
plt.title("Population Mass: Equilibrium (t=1 onward)")
plt.colorbar()
plt.xticks(range(G))
plt.yticks(range(G))
plt.tight_layout()

plt.figure()
plt.imshow(r_eq)
plt.title("Equilibrium Rent per Cell")
plt.colorbar()
plt.xticks(range(G))
plt.yticks(range(G))
plt.tight_layout()

plt.figure()
plt.plot(range(1, T + 1), u_path, marker="o")
plt.title("Utility Level $u_t^*$ over Time")
plt.xlabel("t")
plt.ylabel("$u_t^*$")
plt.tight_layout()

plt.figure()
plt.plot(range(1, T + 1), [rp.mean() for rp in r_path], marker="o")
plt.title("Mean Rent over Time")
plt.xlabel("t")
plt.ylabel("Mean rent")
plt.tight_layout()

In [ ]:
summary = pd.DataFrame(
    {
        "metric": [
            "Occupied cells",
            "Vacant cells",
            "Mass in occupied cells",
            "Mass in vacant cells",
        ],
        "value": [occupied_cells, vacant_cells, mass_occupied, mass_vacant],
    }
)

print("Occupancy summary (equilibrium):")
print(summary)